<a href="https://colab.research.google.com/github/ycaparicioa/MetNumUN2023I/blob/main/Lab8/Week2QRdecompositionGroup2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{I} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [2]:

def householder(vec):
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("Input vector must be one-dimensional")

    n = vec.shape[0]
    outvec = np.zeros(n)
    outvec[0] = np.linalg.norm(vec)

    u = vec - outvec
    u_norm = np.linalg.norm(u)
    if u_norm != 0:
        u /= u_norm

    H = np.eye(n) - 2 * np.outer(u, u)
    outvec = H.dot(vec)

    return outvec, H


Test your function using tests below:

In [3]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [4]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

$$
\mathbf{A} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} \mathbf{R} \;,
$$
so 
$$
\mathbf{A} =  \mathbf{Q} \mathbf{R} \;,
$$
with
$$
\mathbf{Q} = (\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1)^{-1} =  \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T 
$$
Since $\mathbf{H}_i$ is ortogonal
$$
\mathbf{H}_i\mathbf{H}_i^T = \mathbf{I}
$$
then
$$
\mathbf{H}^{-1} = \mathbf{H}^T 
$$
but also  $\mathbf{H}_i$ is symetric
$$
\mathbf{H}_i^T = \mathbf{H}_i
$$
so
$$
\mathbf{Q} = \mathbf{H}_1^{-1}  \mathbf{H}_2^{-1}  \cdots \mathbf{H}_{n-1}^{-1} = \mathbf{H}_1^T  \mathbf{H}_2^T  \cdots \mathbf{H}_{n-1}^T =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$


In [5]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    Q : ndarray, shape(m, m)
        The orthogonal matrix
    R : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    m, n = a.shape
    Q = np.eye(m)
    R = a.copy()

    for j in range(min(m, n)):
        x = R[j:, j]
        norm_x = np.linalg.norm(x)
        if norm_x == 0:
            continue
        y = np.zeros_like(x)
        y[0] = norm_x
        u = (x - y) / np.linalg.norm(x - y)
        H = np.eye(m)
        H[j:, j:] -= 2 * np.outer(u, u)
        R = np.dot(H, R)
        Q = np.dot(Q, H.T)

    return Q,R

In [6]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [7]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your q and r agree with qq and rr. Explain.

In [9]:
from scipy.linalg import qr
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])
q, r = qr_decomp(a)
qq, rr = qr(a)
print(q)
print(qq)
print(r)
print(rr)
print(a)
print(q@r)
print(qq@rr)

[[ 0.27088608  0.02504403 -0.06535136  0.95964669 -0.0283012 ]
 [ 0.30379747  0.58701626 -0.33121325 -0.14303499 -0.65799527]
 [ 0.54683544  0.50315975  0.37333551 -0.12611581  0.54084809]
 [ 0.48607595 -0.47519609  0.56191158 -0.10012449 -0.46059669]
 [ 0.54683544 -0.41928945 -0.65643199 -0.1808013   0.2481437 ]]
[[-0.24479602 -0.06723049  0.00981821 -0.41631113 -0.87300837]
 [-0.30599503 -0.58266423  0.33762629  0.64971343 -0.17535786]
 [-0.55079106 -0.4964713  -0.36457628 -0.41134781  0.38473703]
 [-0.48959205  0.47923271 -0.55563068  0.4547994  -0.1227505 ]
 [-0.55079106  0.42406923  0.66653641 -0.16884307  0.20979928]]
[[16.33417722 10.47341772  4.40253165]
 [-0.01147851  6.50442321 -0.78555051]
 [ 0.02995271 -0.          2.64586952]
 [-0.43983807 -0.          0.        ]
 [ 0.01297139 -0.         -0.        ]]
[[-16.34013464 -10.46503005  -4.40632844]
 [  0.          -6.51790964   0.7843557 ]
 [  0.           0.          -2.63989693]
 [  0.           0.           0.        ]
 [  

Enter your explanation here (10% of the total grade, peer-graded)

**Explanation:**

The QR decomposition is a process by which a matrix is decomposed into the product of an orthogonal matrix Q and an upper triangular matrix R. Mathematically, this is expressed as A=QR.

The QR decomposition is not unique, which means that for a given matrix A, there are multiple valid pairs of matrices Q and R that satisfy the above equation. This is one of the reasons why the Q and R matrices we obtain from our function are not exactly the same as the ones we obtain from the scipy qr function.

In our implementation, we use the Householder transformation to perform the QR decomposition. The Householder transformation is a way to reflect a vector about a plane. In the context of QR decomposition, we use the Householder transformation to "rotate" the columns of matrix A throughout the decomposition process, thus generating the orthogonal matrix Q and the upper triangular matrix R.


We can see that q@r returns the original matrix A, indicating that the QR decomposition was performed correctly. Although the Q and R matrices differ in the signs of some rows compared to the scipy implementation, the decomposition is valid and correct.

In summary, although the implementation of QR decomposition using the Householder transformation may yield results that differ in the signs of the rows compared to other implementations, the result is still a valid and correct QR decomposition of the original matrix.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).





Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

$$
\mathbf{R} = \mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} 
$$
and 
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so if
$$
\mathbf{R}_0 =  \mathbf{A}
$$
then 
$$
\mathbf{R}_1 = \mathbf{H}_1 \mathbf{R}_0= \ (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T) \mathbf{R}_0 =  \mathbf{R}_0 -  2 \mathbf{u}_1 ( \mathbf{u}_1^T  \mathbf{R}_0)
$$
and
$$
\mathbf{R}_2 = \mathbf{H}_2 \mathbf{R}_1 = \ (\mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T) \mathbf{R}_1 =  \mathbf{R}_1 -  2 \mathbf{u}_2 ( \mathbf{u}_2^T  \mathbf{R}_1)
$$
so on until
$$
\mathbf{R} = \mathbf{H}_n \mathbf{R}_{n-1} =  (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T) \mathbf{R}_{n-1} = \mathbf{R}_{n-1} -  2 \mathbf{u}_{n-1} ( \mathbf{u}_{n-1}^T  \mathbf{R}_{n-1})
$$

In [15]:
def r_decomp(a):
    """Compute R without ever forming the  H  matrices
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    
    R : ndarray, shape(m, n)
        The upper triangular matrix
    vecs:  reflection vectors.
        
    """
    # Initialize the dimensions of the matrix and a copy of the input matrix
    m, n = a.shape
    R = np.array(a, copy=True, dtype=float)
    vecs = []
    
    # Loop through each column in the matrix
    for i in range(min(m, n)):
        # Determine x as the ith column from the ith row to the end
        x = R[i:, i]
        
        # Create the first elementary vector with the norm of x
        e1 = np.zeros_like(x)
        e1[0] = np.linalg.norm(x)
        
        # Compute the reflection vector u
        u = (x - e1) / np.linalg.norm(x - e1)
        
        # Make sure u is 2D for outer product
        u = u.reshape(-1, 1)
        
        # Update R using the Householder transformation
        R[i:, :] -= 2 * np.outer(u, np.dot(u.T, R[i:, :]))
        
        # Store the reflection vector
        vecs.append(u)
        
    return R, vecs

$$
\mathbf{Q} =  \mathbf{H}_1 \mathbf{H}_2  \cdots \mathbf{H}_{n-1}
$$
and
$$
\mathbf{H}_i = \mathbf{I} - 2 \mathbf{u}_i \mathbf{u}_i^T
$$
so 
$$
\mathbf{Q} =  (\mathbf{I} - 2 \mathbf{u}_1 \mathbf{u}_1^T )( \mathbf{I} - 2 \mathbf{u}_2 \mathbf{u}_2^T)  \cdots (\mathbf{I} - 2 \mathbf{u}_{n-1} \mathbf{u}_{n-1}^T)
$$

In [16]:
def q_decomp(vecs):
    """Compute Q and QT from reflection vectors
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    
    Q : ndarray, shape(m, m)
        The ortogonal matrix
    Q.T : ndarray, shape(m, m)
        The transpose of Q

    """
    
    # Initialize the dimensions of the matrix and the orthogonal matrix Q
    m, n = len(vecs[0]), len(vecs)
    Q = np.eye(m)

    # Loop through each reflection vector in reverse order
    for i in range(n-1, -1, -1):
        # Retrieve the reflection vector
        u = vecs[i]
        
        # Apply the Householder transformation to compute Q
        Q[i:, :] -= 2 * np.outer(u, np.dot(u.T, Q[i:, :]))

    return Q, Q.T

In [17]:
rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
r, vecs = r_decomp(a)
q,qt = q_decomp(vecs)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

In [18]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

In [19]:
a = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4], [9,3,1]])

R, vecs = r_decomp(a)
print("R\n ",R)
Q,QT = q_decomp(vecs)
print("Q\n",Q)
print("QT\n",QT)
print("QT@Q\n",QT@Q)
print("a\n",a)
print("Q@R\n",Q@R)

R
  [[16.34013464 10.46503005  4.40632844]
 [ 0.          6.51790964 -0.7843557 ]
 [ 0.         -0.          2.63989693]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]
Q
 [[ 0.24479602  0.06723049 -0.00981821  0.96611775  0.04555281]
 [ 0.30599503  0.58266423 -0.33762629 -0.09006618 -0.6669078 ]
 [ 0.55079106  0.4964713   0.36457628 -0.19531237  0.52828277]
 [ 0.48959205 -0.47923271  0.55563068 -0.06304632 -0.46683546]
 [ 0.55079106 -0.42406923 -0.66653641 -0.12799536  0.23694072]]
QT
 [[ 0.24479602  0.30599503  0.55079106  0.48959205  0.55079106]
 [ 0.06723049  0.58266423  0.4964713  -0.47923271 -0.42406923]
 [-0.00981821 -0.33762629  0.36457628  0.55563068 -0.66653641]
 [ 0.96611775 -0.09006618 -0.19531237 -0.06304632 -0.12799536]
 [ 0.04555281 -0.6669078   0.52828277 -0.46683546  0.23694072]]
QT@Q
 [[ 1. -0. -0.  0. -0.]
 [-0.  1. -0.  0. -0.]
 [-0. -0.  1.  0. -0.]
 [ 0.  0.  0.  1.  0.]
 [-0. -0. -0.  0.  1.]]
a
 [[4 3 1]
 [5 7 0]
 [9 9 3]
 [8 2 4]
